# 매출데이터 - 지하철 유동인구 데이터 연결 링크 생성
---
## 개요
매출데이터-상권_코드_명에 '지하철역'을 포함하는 데이터와 1~8호선까지의 지하철 승하차 인원 유동인구 데이터를 연결할 링크 생성

## 사용 파일
서울교통공사_역별 일별 시간대별 승하차인원_20211231.csv, 서울특별시 열린 데이터 광장, 서율교통공사 제공

서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2021년.csv, 서울특별시 열린 데이터 광장, 제공

## 구동 시퀀스
- 매출 데이터에서 '역'을 포함하는 데이터 별도 추출
- 매출 데이터 '상권_코드_명'에서 지하철역 이름만 분리 = 이하 매출역
- 승하차 인원 데이터에서 '하차' 분리 후 '역명'에서 주표기명 분리 = 이하 하차역
- "하차역" 주표기명을 기준으로 하여, "매출역"과 매칭, 매칭된 데이터 별도 저장 = 이하 매칭역
- "하차역"의 주표기명을 사용하여 지하철역 dict(JSON) 생성, (역이름:Num)의 형태
- "매칭역"과 지하철역 dict를 사용하여, link 생성
- "하차역"과 지하철역 dict를 사용하여, link 생성

# 모듈 import 및 데이터 로딩

In [1]:
# Basic Import
import numpy as np
import pandas as pd

In [2]:
# Dataset loading
money_data = pd.read_csv("./data/2021_money_add.csv", encoding="EUC-KR", header=0)  # 자치구, 행정동 포함된 매출 데이터
train_data = pd.read_csv("./data/2021_subway_transport_data.csv", encoding="EUC-KR", header=0)  # 승하차 인원 데이터, 레이블만 뽑을것이기에 하차만 있어도 상관없음

In [3]:
# Loading test
# 주석제거하고 사용
# money_data.head()
# train_data.head()

In [5]:
# Data length check
print(len(money_data[money_data['상권_코드_명'].str.contains("역")]))

54448


# 데이터 필터링 - link 데이터 생성

## 매출 데이터 역 이름 고유 리스트 작성

In [ ]:
# Data filtering
# 매출데이터 내부, 상권코드명에 '역'이 포함된 데이터에서 상권명 고유 값 가져오기 (중복 제거)
station_money = pd.DataFrame(money_data[money_data['상권_코드_명'].str.contains("역")]['상권_코드_명'].unique())
station_money_add = pd.DataFrame(money_data[(money_data['상권_코드_명'].str.contains("DMC"))]['상권_코드_명'].unique())  # DMC(디지털미디어시티) 추가, '역'이 없어서 필터링 되지 않았음
station_money_add2 = pd.DataFrame(money_data[(money_data['상권_코드_명'].str.contains("버티고개"))]['상권_코드_명'].unique())  # 버티고개 추가, '역'이 없어서 필터링 되지 않았음

station_money = pd.concat([station_money, station_money_add], axis=0, ignore_index=True)  # 기존 데이터에 결합
station_money = pd.concat([station_money, station_money_add2], axis=0, ignore_index=True)  # 기존 데이터에 결합
station_money.columns = ["상권코드명",]

# 원본데이터-상권명의 역 이름 위치가 제각각임. 따라서 이름을 찾기위해 분리작업
col_arr = []  # 분리한 횟수 컬럼명 저장용

# 상권_코드_명에서 실제 역이름 분리하기, 정규표현식 사용
splt = station_money["상권코드명"].str.split(r"역$|역 |역\(|역,|역_|역골|역지|역전|역1|\(| |역상|역둘|역동|역\)|역교", expand=True)

# 분리된 데이터의 길이를 확인하여 컬럼명을 생성하고, 이를 적용
for i in range(len(splt.columns)):
    col_name = str(i+1) + "번 분리 데이터" 
    col_arr.append(col_name)
splt.columns = col_arr

# 원본 보존용 copy
origin_station_money = station_money.copy()

# 기존 고유값에, 분리한 데이터를 합치기
# 원본의 이름과, 분리된 이름을 동시에 보고, 나중에 원본명과 링크를 편하게 추출하기 위함
station_money = pd.concat([station_money, splt], axis=1)

## 승하차데이터 역 이름 고유 리스트 작성

In [6]:
# 매출데이터와 동일한 과정으로 지하철 유동인구 데이터도 필터링
station_train = pd.DataFrame(train_data['역명'].unique())
station_train.columns = ["역명",]

col_arr_train = []

splt_train = station_train["역명"].str.split(r"\(|역$", expand=True)

for i in range(len(splt_train.columns)):
    col_name = str(i+1) + "번 분리 데이터"
    col_arr_train.append(col_name)
splt_train.columns = col_arr_train

origin_station_train = station_train.copy()

station_train = pd.concat([station_train, splt_train], axis=1)

True

## 역이름 필터링-매칭

In [ ]:
# 승하차 데이터의 주기역명과 매출데이터 역 이름 필터링-매칭

stt_arr = station_train['1번 분리 데이터'].unique()  # 주기역명 고유값

# pop 사용, 실제 pop는 column 단위 동작이므로, concat으로 pop 효과 내기
poped_df = pd.DataFrame()

for i in range(len(stt_arr)):
    if stt_arr[i] == '둔촌동':  # 역명에는 둔촌동이 나오지만, 매출역에는 둔촌동, 둔촌이 데이터셋에 함께 나오기 때문에 사용
        stt_arr[i] = '둔촌' or '둔촌동'  # 변수 선언 및 값 입력에 or를 사용하면, 제일 처음 나오는 True가 변수로써 사용된다.
        
        # 위의 경우는 역명에 둔촌동이 나오면, 둔촌이나 둔촌동 둘 중 하나로 사용된다를 명시
        # 매출역이 둔촌동인 경우에, 뒤의 '둔촌동'이 True가 되어 사용된다.

    for j in range(len(station_money.columns)):  # 컬럼의 길이에서 loop
        for k in range(len(station_money)):
            if station_money.iloc[k, j] == stt_arr[i]:
                poped = pd.DataFrame(station_money.iloc[k]).T  # Series로 만들어지기 때문에, T를 사용하여 column 유지
                poped['idx'] = j  # 몇 번째 분리 데이터에서 매칭되는지 체크하여 새 컬럼에 입력
                poped_df = pd.concat([poped_df, poped], axis=0, ignore_index=True)  # 매칭데이터들 별도 데이터 프레임에 결합
                station_money.drop([station_money.index[k]], inplace=True, axis=0)  # 매출데이터에서 매칭데이터 제거
                break

# 예상 구동시간 20초

In [ ]:
# 분리 작업 반복 시행 - 검증
# break로 인하여 첫 매칭에 종료되기 때문에, 이후에 매칭되는 데이터가 남는다.
# 이를 걸러주기 위하여 추가적인 반복 시행

prelen = 0  # 필터링-매칭 시행전 길이
cnt = 0  # 반복횟수
loops = 0  # 루프 횟수

while len(station_money) != prelen or cnt < 3:  # 이전 회차와 현재 회차의 결과가 같으면서, 동시에 반복횟수가 3 이상이면 종료
    if loops != 0:  # 최초 시행시에는 시행가 없기에, != 을 사용하여 조건문 사용
        prelen = len(station_money)

    for i in range(len(stt_arr)):
        if stt_arr[i] == '둔촌동':
            stt_arr[i] = '둔촌' or '둔촌동'

        for j in range(len(station_money.columns)):  # 이전 필터링-매칭 동작 반복
            for k in range(len(station_money)):
                if station_money.iloc[k, j] == stt_arr[i]:
                    poped = pd.DataFrame(station_money.iloc[k]).T
                    poped['idx'] = j
                    poped_df = pd.concat([poped_df, poped], axis=0, ignore_index=True)
                    station_money.drop([station_money.index[k]], inplace=True, axis=0)
                    break
    
    if prelen == len(station_money):  # 이전 길이와 현재 길이가 동일하면, 필터링-매칭되어 삭제되지 않았다는 의미
        cnt += 1  # 따라서, 해당 회차 결과 반복 횟수를 +1
    else:
        cnt = 0  # 다른 경우, 이전의 결과와 다르기 때문에 초기화

    loops += 1  # 결과와 관계없이 루프 횟수 증가

# 예상 구동시간 60초

In [ ]:
# 역명:순번 순으로 dict 형 생성, link 컬럼의 값에 사용됨
# key : 역명, value : 순번
stt_dict = dict(zip(stt_arr, range(len(stt_arr))))

## 필터링-매칭 결과 별도 추출

### 매출 데이터에서 매칭된 데이터 연결 링크 리스트 생성

원 데이터셋 명칭과 link 번호를 별도로 추출하여, 다른 매출 데이터에서도 사용 가능하도록 csv로 추출

In [ ]:
# 매칭된 데이터, link 컬럼 추가하기
# 원본 보존
poped_df_origin = poped_df.copy()

# 초기화
poped_df['link'] = None

# 각 개별 행에 고유 링크값 입력
for i in range(len(poped_df)):
    col_idx = poped_df.iloc[i]['idx']  # 반복문을 줄이기 위해 매칭된 위치를 사용
    linked_data = poped_df.iloc[i, col_idx]  # dict의 key 찾기

    # poped_df.iloc[i, -1] = stt_dict[linked_data]  # link 컬럼
    poped_df.loc[i, -1] = stt_dict[linked_data]  # link 컬럼에 value 입력하여 연결고리 생성

# poped_df는 concat으로 새로 생성된 데이터 프레임이고, ignore_index를 사용하였으므로, index는 자동적으로 0부터 순차시작

In [ ]:
# 매출데이터에 기록되어 있는 상권_코드_명과, link만 별도 추출
linked_df = poped_df.loc[:,['상권코드명', 'link']]
# linked_df.to_csv("./sales_district_name_IDcode.csv", encoding='utf-8-sig')  # 한글이 있기 때문에 encoding='utf-8-sig'를 사용해준다.
# to_csv에 encoding='utf-8-sig'를 사용하면, read_csv시 encoding을 별도로 추가하지 않아도 한글을 읽을 수 있다.

In [ ]:
# 승하차 데이터, link 컬럼 추가
# 매칭 데이터 link 추가와 동일한 방식
stt_df = station_train.copy()

if 'link' in stt_df.columns:
    del stt_df['link']

stt_df['link'] = None

for i in range(len(stt_df)):
    linked_data = stt_df.iloc[i]['1번 분리 데이터']
    
    if linked_data == '둔촌동':
        linked_data = '둔촌' or '둔촌동'
        
    stt_df.iloc[i, -1] = stt_dict[linked_data]

In [ ]:
# 승하차데이터에 기록되어 있는 역명과, link만 별도 추출
linked_stt_df = stt_df[['역명', 'link']]
# linked_stt_df.to_csv("./subway_station_name_IDcode.csv", encoding='utf-8-sig')  # 
